In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-model-for-tabular/lgb_1.csv
/kaggle/input/new-model-for-tabular/xgb_1.csv
/kaggle/input/new-model-for-tabular/1_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [2]:
!pip install xlrd

     |████████████████████████████████| 96 kB 749 kB/s 


In [3]:
!pip install AutoViz

In [4]:
import warnings 
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import numpy
import pandas
import matplotlib.pyplot as plt 
import seaborn as sns

from autoviz.AutoViz_Class import AutoViz_Class

from category_encoders import LeaveOneOutEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from scipy.special import expit
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


Imported AutoViz_Class version: 0.0.81. Call using:
    from autoviz.AutoViz_Class import AutoViz_Class
    AV = AutoViz_Class()
    AV.AutoViz(filename, sep=',', depVar='', dfte=None, header=0, verbose=0,
                            lowess=False,chart_format='svg',max_rows_analyzed=150000,max_cols_analyzed=30)
Note: verbose=0 or 1 generates charts and displays them in your local Jupyter notebook.
      verbose=2 saves plots in your local machine under AutoViz_Plots directory and does not display charts.


In [5]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
sub=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 32 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      300000 non-null  int64  
 1   cat0    300000 non-null  object 
 2   cat1    300000 non-null  object 
 3   cat2    300000 non-null  object 
 4   cat3    300000 non-null  object 
 5   cat4    300000 non-null  object 
 6   cat5    300000 non-null  object 
 7   cat6    300000 non-null  object 
 8   cat7    300000 non-null  object 
 9   cat8    300000 non-null  object 
 10  cat9    300000 non-null  object 
 11  cat10   300000 non-null  object 
 12  cat11   300000 non-null  object 
 13  cat12   300000 non-null  object 
 14  cat13   300000 non-null  object 
 15  cat14   300000 non-null  object 
 16  cat15   300000 non-null  object 
 17  cat16   300000 non-null  object 
 18  cat17   300000 non-null  object 
 19  cat18   300000 non-null  object 
 20  cont0   300000 non-null  float64
 21  cont1   30

In [7]:
# AV=AutoViz_Class()
# dftc=AV.AutoViz(filename='',dfte=train,verbose=1,depVar='target')

In [8]:
drop_var=['id', 'cat5', 'cat7', 'cat8', 'cat10'] # Found Out using AutoViz
train.drop(columns=drop_var,inplace=True)
test.drop(columns=drop_var,inplace=True)

In [9]:
cont_features = [
    "cont0", "cont1", "cont2", "cont3", "cont4", "cont5", "cont6", "cont7",
    "cont8", "cont9", "cont10",
]
cat_features = [
    "cat0", "cat1", "cat2", "cat3", "cat4", "cat6", "cat9","cat11",
    "cat12", "cat13", "cat14", "cat15","cat16", "cat17", "cat18"
]
target = train["target"]

In [10]:
sc=StandardScaler()
train[cont_features]=sc.fit_transform(train[cont_features])
test[cont_features]=sc.transform(test[cont_features])

In [11]:
for i in cat_features:
    print(f'The Number of Unique Variable in {i} are {train[i].nunique()}')

The Number of Unique Variable in cat0 are 2
The Number of Unique Variable in cat1 are 15
The Number of Unique Variable in cat2 are 19
The Number of Unique Variable in cat3 are 13
The Number of Unique Variable in cat4 are 20
The Number of Unique Variable in cat6 are 16
The Number of Unique Variable in cat9 are 19
The Number of Unique Variable in cat11 are 2
The Number of Unique Variable in cat12 are 2
The Number of Unique Variable in cat13 are 2
The Number of Unique Variable in cat14 are 2
The Number of Unique Variable in cat15 are 4
The Number of Unique Variable in cat16 are 4
The Number of Unique Variable in cat17 are 4
The Number of Unique Variable in cat18 are 4


In [12]:
xgb_cat_features = []
lgb_cat_features = []
cb_cat_features = []
ridge_cat_features = []
sgd_cat_features = []
hgbc_cat_features = []

loo_features = []
le_features = []

def le_encoder(train_df, test_df, column):
    le = LabelEncoder()
    new_feature = "{}_le".format(column)
    le.fit(train_df[column].unique().tolist() + test_df[column].unique().tolist())
    train_df[new_feature] = le.transform(train_df[column])
    test_df[new_feature] = le.transform(test_df[column])
    return new_feature

def loo_encoder(train_df,test_df,column):
    loo=LeaveOneOutEncoder()
    new_feature=f'{column}_loo'
    loo.fit(train_df[column],train_df['target'])
    train_df[new_feature]=loo.transform(train_df[column])
    test_df[new_feature]=loo.transform(test_df[column])
    return new_feature

for feature in cat_features:
    loo_features.append(loo_encoder(train,test,feature))
    le_features.append(le_encoder(train,test,feature))
    
xgb_cat_features.extend(loo_features)
d
cb_cat_features.extend(cat_features)
ridge_cat_features.extend(loo_features)
# sgd_cat_features.extend(loo_features)
# hgbc_cat_features.extend(loo_features)

NameError: name 'd' is not defined

In [13]:
random_state=101
n_folds=10
k_fold = StratifiedKFold(n_splits=n_folds,random_state=random_state,shuffle=True)



xgb_train_preds=np.zeros(len(train.index),)
xgb_test_preds=np.zeros(len(test.index),)
xgb_features=xgb_cat_features + cont_features

lgb_train_preds = numpy.zeros(len(train.index), )
lgb_test_preds = numpy.zeros(len(test.index), )
lgb_features = lgb_cat_features + cont_features

cb_train_preds = numpy.zeros(len(train.index), )
cb_test_preds = numpy.zeros(len(test.index), )
cb_features = cb_cat_features + cont_features

ridge_train_preds = numpy.zeros(len(train.index), )
ridge_test_preds = numpy.zeros(len(test.index), )
ridge_features = ridge_cat_features + cont_features

sgd_train_preds = numpy.zeros(len(train.index), )
sgd_test_preds = numpy.zeros(len(test.index), )
sgd_features = sgd_cat_features + cont_features

hgbc_train_preds = numpy.zeros(len(train.index), )
hgbc_test_preds = numpy.zeros(len(test.index), )
hgbc_features = hgbc_cat_features + cont_features

for fold,(train_index,test_index) in enumerate(k_fold.split(train,target)):
    print(f"--> Fold {fold+1}")
#     print(train_index)
#     print(test_index)
    y_train=target.iloc[train_index]
    y_valid=target.iloc[test_index]
    
    xgb_x_train = pandas.DataFrame(train[xgb_features].iloc[train_index])
    xgb_x_valid = pandas.DataFrame(train[xgb_features].iloc[test_index])

    lgb_x_train = pandas.DataFrame(train[lgb_features].iloc[train_index])
    lgb_x_valid = pandas.DataFrame(train[lgb_features].iloc[test_index])

    cb_x_train = pandas.DataFrame(train[cb_features].iloc[train_index])
    cb_x_valid = pandas.DataFrame(train[cb_features].iloc[test_index])

#     ridge_x_train = pandas.DataFrame(train[ridge_features].iloc[train_index])
#     ridge_x_valid = pandas.DataFrame(train[ridge_features].iloc[test_index])

#     sgd_x_train = pandas.DataFrame(train[sgd_features].iloc[train_index])
#     sgd_x_valid = pandas.DataFrame(train[sgd_features].iloc[test_index])

#     hgbc_x_train = pandas.DataFrame(train[hgbc_features].iloc[train_index])
#     hgbc_x_valid = pandas.DataFrame(train[hgbc_features].iloc[test_index])
    
    xgb_model = XGBClassifier(
        random_state=0,
        n_estimators=8790,
        verbosity=1,
        eval_metric="auc",
        tree_method="gpu_hist",
        gpu_id=0,
        booster= 'gbtree',
        alpha=0.26389513426946043,
        colsample_bytree=0.5165904538770105,
        gamma=7.272747952404219e-05,
        reg_lambda=0.014576256860713442,
        learning_rate=0.010786623160708754,
        max_bin=338,
        max_depth=17,
        min_child_weight=30,
        subsample=0.737564575608887,
    )
    xgb_model.fit(
        xgb_x_train,
        y_train,
        eval_set=[(xgb_x_valid, y_valid)], 
        verbose=0,
        early_stopping_rounds=200
    )
    train_oof_preds=xgb_model.predict_proba(xgb_x_valid)[:,1]
    test_oof_preds=xgb_model.predict_proba(test[xgb_features])[:,1]
    xgb_train_preds[test_index]=train_oof_preds
    xgb_test_preds+=test_oof_preds/n_folds
    print(f": XGB - ROC AUC SCORE = {roc_auc_score(y_valid,train_oof_preds,average='micro')}")
    
    
    lgb_model = LGBMClassifier(
        cat_feature=[x for x in range(len(lgb_cat_features))],
        random_state=42,
        cat_l2=25.999876242730252,
        cat_smooth=89.2699690675538,
        colsample_bytree=0.34162338483795796,
        early_stopping_round=200,
        learning_rate=0.03376414440763976,
        max_bin=788,
        max_depth=13,
        metric="auc",
#         min_child_samples=41,
#         min_child_weight= 0.00032169039200189445,
        min_data_per_group=177,
        n_estimators=5721,
        n_jobs=-1,
        num_leaves=286,
        reg_alpha=0.8652493397013724,
        reg_lambda=0.028016961044558177,
        subsample=0.28929714816950847,
        subsample_freq=1,
        verbose=-1,
    )
    lgb_model.fit(
        lgb_x_train,
        y_train,
        eval_set=[(lgb_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = lgb_model.predict_proba(lgb_x_valid)[:,1]
    test_oof_preds = lgb_model.predict_proba(test[lgb_features])[:,1]
    lgb_train_preds[test_index] = train_oof_preds
    lgb_test_preds += test_oof_preds / n_folds
    print(": LGB - ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    
    
    cb_model = CatBoostClassifier(
        verbose=0,
        eval_metric="AUC",
        loss_function="Logloss",
        random_state=random_state,
        num_boost_round=20000,
        od_type="Iter",
        od_wait=200,
        task_type="GPU",
        devices="0",
        cat_features=[x for x in range(len(cb_cat_features))],
        bagging_temperature=1.288692494969795,
        grow_policy="Depthwise",
        l2_leaf_reg=9.847870133539244,
        learning_rate=0.01877982653902465,
        max_depth=8,
        min_data_in_leaf=1,
        penalties_coefficient=2.1176668909602734,
    )
    cb_model.fit(
        cb_x_train,
        y_train,
        eval_set=[(cb_x_valid, y_valid)], 
        verbose=0,
    )

    train_oof_preds = cb_model.predict_proba(cb_x_valid)[:,1]
    test_oof_preds = cb_model.predict_proba(test[cb_features])[:,1]
    cb_train_preds[test_index] = train_oof_preds
    cb_test_preds += test_oof_preds / n_folds
    print(": CB - ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))

#     ridge_model = CalibratedClassifierCV(
#         RidgeClassifier(random_state=random_state),
#         cv=3,
#     )
#     ridge_model.fit(
#         ridge_x_train,
#         y_train,
#     )

#     train_oof_preds = ridge_model.predict_proba(ridge_x_valid)[:,-1]
#     test_oof_preds = ridge_model.predict_proba(test[ridge_features])[:,-1]
#     ridge_train_preds[test_index] = train_oof_preds
#     ridge_test_preds += test_oof_preds / n_folds
#     print(": Ridge - ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    
#     sgd_model = CalibratedClassifierCV(
#         SGDClassifier(
#             random_state=random_state,
#             n_jobs=-1,
#             loss="squared_hinge",
#         ),
#         cv=3,
#     )
#     sgd_model.fit(
#         sgd_x_train,
#         y_train,
#     )

#     train_oof_preds = sgd_model.predict_proba(sgd_x_valid)[:,-1]
#     test_oof_preds = sgd_model.predict_proba(test[sgd_features])[:,-1]
#     sgd_train_preds[test_index] = train_oof_preds
#     sgd_test_preds += test_oof_preds / n_folds
#     print(": SGD - ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    
#     hgbc_model = HistGradientBoostingClassifier(
#         l2_regularization=1.766059063693552,
#         learning_rate=0.10675193678150449,
#         max_bins=128,
#         max_depth=31,
#         max_leaf_nodes=185,
#         random_state=2021
#     )
#     hgbc_model.fit(
#         hgbc_x_train,
#         y_train,
#     )

#     train_oof_preds = hgbc_model.predict_proba(hgbc_x_valid)[:,-1]
#     test_oof_preds = hgbc_model.predict_proba(test[hgbc_features])[:,-1]
#     hgbc_train_preds[test_index] = train_oof_preds
#     hgbc_test_preds += test_oof_preds / n_folds
#     print(": HGBC - ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    print("")
    
print("--> Overall metrics")
print(": XGB - ROC AUC Score = {}".format(roc_auc_score(target, xgb_train_preds, average="micro")))
print(": LGB - ROC AUC Score = {}".format(roc_auc_score(target, lgb_train_preds, average="micro")))
print(": CB - ROC AUC Score = {}".format(roc_auc_score(target, cb_train_preds, average="micro")))
# print(": Ridge - ROC AUC Score = {}".format(roc_auc_score(target, ridge_train_preds, average="micro")))
# print(": SGD - ROC AUC Score = {}".format(roc_auc_score(target, sgd_train_preds, average="micro")))
# print(": HGBC - ROC AUC Score = {}".format(roc_auc_score(target, hgbc_train_preds, average="micro")))

--> Fold 1
: XGB - ROC AUC SCORE = 0.8944876647761435
: LGB - ROC AUC Score = 0.8264153798546463
: CB - ROC AUC Score = 0.8290756521455894

--> Fold 2
: XGB - ROC AUC SCORE = 0.8927723364231186
: LGB - ROC AUC Score = 0.8277516874981211
: CB - ROC AUC Score = 0.8295002443605063

--> Fold 3
: XGB - ROC AUC SCORE = 0.8919766554285864
: LGB - ROC AUC Score = 0.8218443227616647
: CB - ROC AUC Score = 0.8250825074786687

--> Fold 4
: XGB - ROC AUC SCORE = 0.8950375073005141
: LGB - ROC AUC Score = 0.8263724275980854
: CB - ROC AUC Score = 0.8290875272147941

--> Fold 5
: XGB - ROC AUC SCORE = 0.8962675755874688
: LGB - ROC AUC Score = 0.8277578276107902
: CB - ROC AUC Score = 0.8307372488063358

--> Fold 6
: XGB - ROC AUC SCORE = 0.8914147295505201
: LGB - ROC AUC Score = 0.8259081472013721
: CB - ROC AUC Score = 0.8267288109219867

--> Fold 7
: XGB - ROC AUC SCORE = 0.8928011652792561
: LGB - ROC AUC Score = 0.8230913648080378
: CB - ROC AUC Score = 0.8254736657529463

--> Fold 8
: XGB - R

In [14]:
# xgb_1=pd.read_csv('/kaggle/input/new-model-for-tabular/xgb_1.csv')
# lgb_1=pd.read_csv('/kaggle/input/new-model-for-tabular/lgb_1.csv')
# automl=pd.read_csv('/kaggle/input/new-model-for-tabular/1_submission.csv')

In [15]:
# xgb_1=xgb_1['target']
# lgb_1=lgb_1['target']
# automl=automl['target']

In [16]:
random_state=2021
n_folds = 10
k_fold = StratifiedKFold(n_splits=n_folds,random_state=random_state,shuffle=True)

l1_train=pd.DataFrame(data={
    'xgb':xgb_train_preds.tolist(),
    'lgb':lgb_train_preds.tolist(),
    'cb':cb_train_preds.tolist(),
#     'ridge':ridge_train_preds.tolist(),
#     'lgb_1':lgb_1.tolist(),
#     'xgb_1':xgb_1.tolist(),
#     'automl':automl.tolist(),
#     'sgd':sgd_train_preds.tolist(),
#     'hgbc':hgbc_train_preds.tolist(),
    'target':target.tolist()
})
l1_test=pd.DataFrame(data={
    'xgb':xgb_test_preds.tolist(),
    'lgb':lgb_test_preds.tolist(),
    'cb':cb_test_preds.tolist(),
#     'ridge':ridge_test_preds.tolist(),
#     'sgd':sgd_test_preds.tolist(),
#     'hgbc':hgbc_test_preds.tolist()
})

train_preds=numpy.zeros(len(l1_train.index))
test_preds=numpy.zeros(len(l1_test.index))
# features=['xgb','lgb','cb','ridge','sgd','hgbc']
features=['xgb','lgb','cb']

for fold,(train_index,test_index) in enumerate(k_fold.split(l1_train,target)):
    print(f"--> Fold {fold+1}")
    y_train=target.iloc[train_index]
    y_valid=target.iloc[test_index]
    
    x_train=pandas.DataFrame(l1_train[features].iloc[train_index])
    x_valid=pandas.DataFrame(l1_train[features].iloc[test_index])
    
    model=CalibratedClassifierCV(
        RidgeClassifier(random_state=random_state),
        cv=3
    )
    model.fit(x_train,y_train)
    train_oof_preds = model.predict_proba(x_valid)[:,-1]
    test_oof_preds = model.predict_proba(l1_test[features])[:,-1]
    train_preds[test_index] = train_oof_preds
    test_preds += test_oof_preds / n_folds
    print(": ROC AUC Score = {}".format(roc_auc_score(y_valid, train_oof_preds, average="micro")))
    print("")
        
print("--> Overall metrics")
print(": ROC AUC Score = {}".format(roc_auc_score(target, train_preds, average="micro")))

# submission = pandas.read_csv("../input/tabular-playground-series-mar-2021/sample_submission.csv")
sub["target"] = test_preds.tolist()
sub.to_csv("submission.csv", index=False)

--> Fold 1
: ROC AUC Score = 0.8956224100964818

--> Fold 2
: ROC AUC Score = 0.8928151288997961

--> Fold 3
: ROC AUC Score = 0.8923988908902206

--> Fold 4
: ROC AUC Score = 0.8921476940875349

--> Fold 5
: ROC AUC Score = 0.893292659613998

--> Fold 6
: ROC AUC Score = 0.8948944985982853

--> Fold 7
: ROC AUC Score = 0.8958474087046849

--> Fold 8
: ROC AUC Score = 0.8949198065905596

--> Fold 9
: ROC AUC Score = 0.8911073044948753

--> Fold 10
: ROC AUC Score = 0.8920077465779602

--> Overall metrics
: ROC AUC Score = 0.8934838019604299
